# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [5]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):  
    file_path_list = glob.glob(os.path.join(root, '*'))


/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [8]:
ull_data_rows_list = [] 
    
for f in file_path_list:
 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile:  
        csvreader = csv.reader(csvfile) 
        next(csvreader)
         
        for line in csvreader:
            ull_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in ull_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [9]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [10]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [11]:
# TO-DO: Create a Keyspace 
session.execute(""" CREATE KEYSPACE IF NOT EXISTS sparkify
                    WITH REPLICATION = 
                    {'class':'SimpleStrategy','replication_factor':1}""")

#### Set Keyspace

In [12]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkify')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [13]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = """
CREATE TABLE IF NOT EXISTS song_play_detail (
    sessionId int,
    itemInSession int,
    artist text,
    song text,
    length float,
    PRIMARY KEY (sessionId, itemInSession)
)
"""
session.execute(query)

query1 = """
CREATE TABLE IF NOT EXISTS user_song_his(
   userId int,
   sessionId int,
   itemInsession int,
   song text,
   artist text,
   firstName text,
   lastName text,
   PRIMARY KEY ((userId,sessionId),itemInSession))"""
session.execute(query1)

query2 = """
CREATE TABLE IF NOT EXISTS song_listen(
    song text,
    userId int,
    firstName text,
    lastName text,
    PRIMARY KEY(song,userId))"""
session.execute(query2)
                    

In [14]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_play_detail (sessionId, itemInSession,artist,song,length)"
        query = query + "VALUES (%s,%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]),line[0],line[9], float(line[5])) )
        
        query1 = "INSERT INTO user_song_his (userId,sessionId,itemInsession,song,artist,firstName,lastName)"
        query1 = query1 + "VALUES(%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query1, (int(line[10]),int(line[8]),int(line[3]),line[9],line[0],line[1],line[4]))
        
        query2 = "INSERT INTO song_listen(song,userId,firstName,lastName)"
        query2 = query2 + "VALUES(%s,%s,%s,%s)"
        session.execute(query2, (line[9], int(line[10]), line[1], line[4]))


#### Do a SELECT to verify that the data have been inserted into each table

In [15]:
rows = session.execute("SELECT artist,song,length FROM song_play_detail WHERE sessionId = 338 AND itemInSession = 4")
for row in rows:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


In [17]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
rows = session.execute("SELECT * FROM user_song_his")
for row in rows:
    print(row)

Row(userid=58, sessionid=768, iteminsession=0, artist='System of a Down', firstname='Emily', lastname='Benson', song='Sad Statue')
Row(userid=58, sessionid=768, iteminsession=1, artist='Ghostland Observatory', firstname='Emily', lastname='Benson', song='Stranger Lover')
Row(userid=58, sessionid=768, iteminsession=2, artist='Evergreen Terrace', firstname='Emily', lastname='Benson', song='Zero')
Row(userid=85, sessionid=776, iteminsession=2, artist='Deftones', firstname='Kinsley', lastname='Young', song='Head Up (LP Version)')
Row(userid=85, sessionid=776, iteminsession=3, artist='The Notorious B.I.G.', firstname='Kinsley', lastname='Young', song='Playa Hater (Amended Version)')
Row(userid=85, sessionid=776, iteminsession=4, artist='Orchestral Manoeuvres In The Dark', firstname='Kinsley', lastname='Young', song='Stay (The Black Rose And The Universal Wheel)')
Row(userid=85, sessionid=776, iteminsession=5, artist='Vangelis', firstname='Kinsley', lastname='Young', song="Roxane's Veil")
Row

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [20]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
rows = session.execute("SELECT artist, song,firstName, lastName FROM user_song_his WHERE userId = 10 AND sessionId = 182")
for row in rows:
    print(row)

                    

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


In [21]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
rows = session.execute("SELECT firstName, lastName FROM song_listen WHERE song = 'All Hands Against His Own'")
for row in rows:
    print(row)
                    

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [22]:
## TO-DO: Drop the table before closing out the sessions
session.execute("DROP TABLE song_play_detail")

In [24]:
session.execute("DROP TABLE song_listen")

InvalidRequest: Error from server: code=2200 [Invalid query] message="unconfigured table song_listen"

In [25]:
session.execute("DROP TABLE user_song_his")

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()